In [ ]:
# for plotting

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

sns.set()
sns.set_context("poster")
sns.set_style("ticks")
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "cmr10"
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
import numpy as np
import xarray as xr
import metpy
from metpy.units import units
from metpy.constants import water_heat_vaporization, dry_air_gas_constant, earth_gravity
from metpy.interpolate import interpolate_1d
from pandas import Timestamp, Timedelta
import netCDF4 as nc
import glob

In [ ]:
def extract_sounding_sonde_pin(sonde_file):
    sonde = xr.open_dataset(sonde_file)
    t = sonde.tdry + 273.15  # deg C to K
    p = sonde.pres * 100.0
    rv = metpy.calc.mixing_ratio_from_relative_humidity(
        sonde.pres.metpy.quantify(),
        sonde.tdry.metpy.magnitude * units("degC"),
        sonde.rh.metpy.quantify(),
    )
    z = sonde.alt
    u = sonde.u_wind
    v = sonde.v_wind
    return [z, p, t, rv, u, v]

In [ ]:
def extract_sfc_fluxes_merra2_pin(d1, d2, t1, t2, lat, lon, dx=2.5):
    t1e = t1 - Timedelta(1, 'H')
    t2e = t2 + Timedelta(1, 'H')
    shf = d1.HFLUX.loc[t1e:t2e, lat - dx : lat + dx, lon - dx : lon + dx].mean(
        axis=(1, 2)
    )
    lhf = d1.EFLUX.loc[t1e:t2e, lat - dx : lat + dx, lon - dx : lon + dx].mean(
        axis=(1, 2)
    )
    sst = d2.TS.loc[t1e:t2e, lat - dx : lat + dx, lon - dx : lon + dx].mean(axis=(1, 2))
    print(
        "Latitude points in the averageing box: "
        + repr(d1.lat.loc[lat - dx : lat + dx].values.tolist())
    )
    print(
        "Longitude points in the averageing box: "
        + repr(d1.lon.loc[lon - dx : lon + dx].values.tolist())
    )
    time = [(t - t1).total_seconds() for t in d1.time.loc[t1e:t2e].values]
    return np.asarray(time), sst, shf, lhf

In [ ]:
def extract_forcing_merra2_pin(d, t1, t2, lat, lon, dx_in = 2.5):
    rd = dry_air_gas_constant
    g = earth_gravity

    dx = 5.0 * dx_in

    p = d.PL.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx].metpy.quantify()
    z = d.H.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx].metpy.quantify()
    t = d.T.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx].metpy.quantify()
    qv = d.QV.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx].metpy.quantify()
    rv = metpy.calc.mixing_ratio_from_specific_humidity(qv)
    u = d.U.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx].metpy.quantify()
    v = d.V.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx].metpy.quantify()
    omega = d.OMEGA.loc[
        t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx
    ].metpy.quantify()
    tm = t.metpy.quantify() * (1.0 + 0.609133 * rv.metpy.quantify())
    p = p.metpy.assign_crs(
        grid_mapping_name="latitude_longitude", earth_radius=6371229.0
    )
    z = z.metpy.assign_crs(
        grid_mapping_name="latitude_longitude", earth_radius=6371229.0
    )

    # calculate geostrophic winds
    lnp = np.log(p.metpy.dequantify())
    lnp.attrs["units"] = ""
    dpx1, dpy1 = metpy.calc.geospatial_gradient(z * g)
    dpx2, dpy2 = metpy.calc.geospatial_gradient(lnp)
    dpx = dpx1 + rd * tm.metpy.quantify() * dpx2
    dpy = dpy1 + rd * tm.metpy.quantify() * dpy2
    f = metpy.calc.coriolis_parameter(lat * units("degrees"))
    vg = dpx / f
    ug = -dpy / f

    w = -omega.metpy.quantify() * rd * tm.metpy.quantify() / p.metpy.quantify() / g
    # print(omega.metpy.units)
    # print(tm.metpy.units)
    # print(rd.units)
    # print(p.metpy.units)
    # print(w.metpy.units)

    # calculate horizontal advection tendencies
    tp = metpy.calc.potential_temperature(p, t)
    ma = metpy.calc.advection(rv, u, v)
    ta = metpy.calc.advection(tp, u, v)

    time = [(t - t1).total_seconds() for t in d.time.loc[t1:t2].values]

    dx = dx_in
    print(
        "Latitude points in the averageing box: "
        + repr(d.lat.loc[lat - dx : lat + dx].values.tolist())
    )
    print(
        "Longitude points in the averageing box: "
        + repr(d.lon.loc[lon - dx : lon + dx].values.tolist())
    )
    return (
        np.asarray(time),
        (
            d.PHIS.loc[t1:t2, lat - dx : lat + dx, lon - dx : lon + dx]
            * units("m**2/s**2")
            / g
        ).metpy.dequantify(),
        d.PS.loc[t1:t2, lat - dx : lat + dx, lon - dx : lon + dx],
        z.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
        p.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
        w.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
        d.OMEGA.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
        u.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
        v.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
        ta.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
        ma.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
        ug.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
        vg.loc[t1:t2, :, lat - dx : lat + dx, lon - dx : lon + dx],
    )

In [ ]:
def interpolate_forcing_merra2(zout, z, w, u, v, ta, ma, ug, vg, zs):
    """
    interpolate all the forcing profiles to SAM's vertical grid given by `zout`
    """

    nt, nz, ny, nx = z.shape
    zi = np.zeros((nt, nz + 1, ny, nx))
    """
    This is just one way to take into account surface elevation.
    The other way is to interpolate using z (heights above MSL) directly,
    then add a mean surface elevation to zout. Using the latter method, the averaging
    will be done on heights above MSL rather than heights above surface.
    But given the small surface elevations we have around ENA, it shouldn't make much difference. 
    """
    zi[:, :-1, :, :] = z.values - zs.values[:, np.newaxis, :, :]
    # zi[:, :-1, :, :] = z.values
    # zout = zout + zs.values.mean()
    out = []
    sfc = [0, 0, 0, 1, 1, 1, 1]
    for v, s in zip([w, u, v, ta, ma, ug, vg], sfc):
        vi = np.zeros_like(zi)
        vi[:, :-1, :, :] = v.values
        # surface winds are all zero
        if s == 1:
            vi[:, -1, :, :] = v.values[:, -1, :, :]
        # vi[:, -1, :, :] = v.values[:, -1, :, :]
        vo = interpolate_1d(zout, zi, vi, axis=1)
        out.append(vo.mean(axis=(2, 3)))
    return out

In [ ]:
zs_snd = 30.0  # surface elevation of ENA site in m
domain_height = 6500.0  # model top
dz_out = 25.0  # m
z_out = np.arange(dz_out * 0.5, domain_height + dz_out, dz_out)
lat = 39.0916  # deg N
lon = -28.0257  # deg E
lon_w = - lon # deg W, used by the solar zenith angle calculation method in the RRTM wrapper
forc_dir = "/ccsopen/home/hengxiao80/proj/ena_forcing_check/forcing"
sonde_dir = "/ccsopen/home/hengxiao80/proj/ena_forcing_check/obs/enasondewnpnC1"
# start_time = Timestamp("2016-10-22 00:00:00")
# end_time = Timestamp("2016-10-23 00:00:00")
# start_time = Timestamp("2018-11-21 00:00:00")
# end_time = Timestamp("2018-11-22 00:00:00")
start_time = Timestamp("2018-10-28 00:00:00")
end_time = Timestamp("2018-10-29 00:00:00")
start_doy = start_time.dayofyear
start_hour = start_time.hour

In [ ]:
outdate = start_time.strftime("%Y%m%d_%H%M")
outfile = f"{outdate}.merra2.nc"
outroot = nc.Dataset(outfile, "w")

In [ ]:
snd_time_adj = start_time - Timedelta(1, "h")
snd_datestring = snd_time_adj.strftime("%Y%m%d.%H")
snd_file = glob.glob(f"{sonde_dir}/enasondewnpnC1.b1.{snd_datestring}*.cdf")[0]
z_snd, p_snd, t_snd, rv_snd, u_wind_snd, v_wind_snd = extract_sounding_sonde_pin(
    snd_file
)

In [ ]:
init_group = outroot.createGroup("initialization")
init_group.createDimension("z", len(z_snd))
init_group.createDimension("const", 1)

surfp = init_group.createVariable("surface_pressure", "f8", ("const"))
surfp[:] = p_snd[0]  # p_srf
surft = init_group.createVariable("surface_temperature", "f8", ("const"))
surft[:] = t_snd[0]  # t_srf

u_in_domain = u_wind_snd[z_snd < domain_height]
u00 = 0.5 * (np.amin(u_in_domain) + np.amax(u_in_domain))
v_in_domain = v_wind_snd[z_snd < domain_height]
v00 = 0.5 * (np.amin(v_in_domain) + np.amax(v_in_domain))
u0 = init_group.createVariable("reference_u0", "f8", ("const"))
u0[:] = u00
v0 = init_group.createVariable("reference_v0", "f8", ("const"))
v0[:] = v00

z_ini = init_group.createVariable("z", "f8", ("z"))
z_ini[:] = z_snd[:] - zs_snd
pres_ini = init_group.createVariable("pressure", "f8", ("z"))
pres_ini[:] = p_snd[:]
t_ini = init_group.createVariable("temperature", "f8", ("z"))
t_ini[:] = t_snd[:]
qv_ini = init_group.createVariable("vapor_mixing_ratio", "f8", ("z"))
qv_ini[:] = rv_snd.magnitude[:]
u_ini = init_group.createVariable("u", "f8", ("z"))
u_ini[:] = u_wind_snd[:]
v_ini = init_group.createVariable("v", "f8", ("z"))
v_ini[:] = v_wind_snd[:]

In [ ]:
datem1 = (start_time - Timedelta(1, "D")).strftime("%Y%m%d")
date0 = start_time.strftime("%Y%m%d")
datep1 = (start_time + Timedelta(1, "D")).strftime("%Y%m%d")
merra2_l = xr.open_mfdataset(
    [
        f"{forc_dir}/merra2/data/MERRA2_400.inst3_3d_asm_Nv.{datem1}.nc4",
        f"{forc_dir}/merra2/data/MERRA2_400.inst3_3d_asm_Nv.{date0}.nc4",
        f"{forc_dir}/merra2/data/MERRA2_400.inst3_3d_asm_Nv.{datep1}.nc4",
    ]
)
merra2_sfc = xr.open_mfdataset(
    [
        f"{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.{datem1}.nc4",
        f"{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.{date0}.nc4",
        f"{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.{datep1}.nc4",
    ]
)
merra2_sfc2 = xr.open_mfdataset(
    [
        f"{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.{datem1}.nc4",
        f"{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.{date0}.nc4",
        f"{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.{datep1}.nc4",
    ]
)

In [ ]:
time_sfc_merra2, sst_merra2, shf_merra2, lhf_merra2 = extract_sfc_fluxes_merra2_pin(
    merra2_sfc, merra2_sfc2, start_time, end_time, lat, lon
)

In [ ]:
print(time_sfc_merra2)
time_sfc_merra2_i = np.linspace(0., 3600.*24, 25)
print(time_sfc_merra2_i)

In [ ]:
sfc = outroot.createGroup("surface")
sfc.createDimension("t", len(time_sfc_merra2_i))

time_sfc = sfc.createVariable("times", "f8", ("t"))
time_sfc[:] = time_sfc_merra2_i[:]

shf = sfc.createVariable("sensible_heat_flux", "f8", ("t"))
shf[:] =  np.interp(time_sfc_merra2_i, time_sfc_merra2, shf_merra2)
lhf = sfc.createVariable("latent_heat_flux", "f8", ("t"))
lhf[:] =  np.interp(time_sfc_merra2_i, time_sfc_merra2, lhf_merra2)
tsk = sfc.createVariable("skin_temperature", "f8", ("t"))
tsk[:] =  np.interp(time_sfc_merra2_i, time_sfc_merra2, sst_merra2)
ustar = sfc.createVariable("friction_velocity", "f8", ("t"))
ustar[:] = np.ones(len(time_sfc_merra2_i)) * 0.25

In [ ]:
time_merra2, zs, ps, z, p, w, omega, u, v, ta, ma, ug, vg = extract_forcing_merra2_pin(
    merra2_l, start_time, end_time, lat, lon
)
(
    w_merra2,
    u_merra2,
    v_merra2,
    ta_merra2,
    ma_merra2,
    ug_merra2,
    vg_merra2,
) = interpolate_forcing_merra2(z_out, z, w, u, v, ta, ma, ug, vg, zs)
del zs, ps, z, p, w, omega, u, v, ta, ma, ug, vg

In [ ]:
print(time_merra2)

In [ ]:
forcing_group = outroot.createGroup("forcing")

forcing_group.createDimension("t", len(time_merra2))
forcing_group.createDimension("z", len(z_out))
forcing_group.createDimension("const", 1)

time_lsf = forcing_group.createVariable("times", "f8", ("t"))
time_lsf[:] = time_merra2[:]
z_lsf = forcing_group.createVariable("z", "f8", ("z"))
z_lsf[:] = z_out[:]
lat_lsf = forcing_group.createVariable("latitude", "f8", ("const"))
lat_lsf[:] = lat

ug_lsf = forcing_group.createVariable("u_geostrophic", "f8", ("t", "z"))
ug_lsf[:, :] = ug_merra2[:, :]
vg_lsf = forcing_group.createVariable("v_geostrophic", "f8", ("t", "z"))
vg_lsf[:, :] = vg_merra2[:, :]
ur_lsf = forcing_group.createVariable("u_relaxation", "f8", ("t", "z"))
ur_lsf[:, :] = u_merra2[:, :]
vr_lsf = forcing_group.createVariable("v_relaxation", "f8", ("t", "z"))
vr_lsf[:, :] = v_merra2[:, :]
w_lsf = forcing_group.createVariable("subsidence", "f8", ("t", "z"))
w_lsf[:, :] = w_merra2[:, :]
tha_lsf = forcing_group.createVariable("theta_advection", "f8", ("t", "z"))
tha_lsf[:, :] = ta_merra2[:, :]
qva_lsf = forcing_group.createVariable("qt_advection", "f8", ("t", "z"))
qva_lsf[:, :] = ma_merra2[:, :]
# not doing relaxation for thermodynamics for now
t_lsf = forcing_group.createVariable("th_relaxation", "f8", ("t", "z"))
t_lsf[:, :] = 0.0
qv_lsf = forcing_group.createVariable("qt_relaxation", "f8", ("t", "z"))
qv_lsf[:, :] = 0.0

In [ ]:
# Radiation profile (z,p) set in same way as PINACLES reference

h_rad = np.logspace(3.5, 4.3)
p_rad = np.interp(h_rad, z_snd - zs_snd, p_snd)
# print(p_rad)
# print(h_rad)
# plt.plot(p_rad, h_rad, "o")

In [ ]:
# coszrs = 1.0
# aldir = (0.026 / (coszrs**1.7 + 0.065)) + (
#     0.15 * (coszrs - 0.10) * (coszrs - 0.50) * (coszrs - 1.00)
# )
# print(aldir)

In [ ]:
rad_group = outroot.createGroup("radiation")
rad_group.createDimension("layers", len(h_rad))
rad_group.createDimension("const", 1)

radlat = rad_group.createVariable("latitude", "f8", ("const"))
radlat[:] = lat
radlon = rad_group.createVariable("longitude", "f8", ("const"))
radlon[:] = lon_w
day_year = rad_group.createVariable("day_of_year", "f8", ("const"))
day_year[:] = start_doy * 1.0
hour_utc = rad_group.createVariable("hour_utc", "f8", ("const"))
hour_utc[:] = start_hour * 1.0
emissivity = rad_group.createVariable("emissivity", "f8", ("const"))
emissivity[:] = 0.95 # Value from SAM RRTMG
albedo = rad_group.createVariable("albedo", "f8", ("const"))
albedo[:] = 0.06 # Value for diffusive albedeo in SAM RRTMG
h = rad_group.createVariable("z", "f8", ("layers"))
h[:] = h_rad[:]
p = rad_group.createVariable("pressure", "f8", ("layers"))
p[:] = p_rad[:]
t = rad_group.createVariable("temperature", "f8", ("layers"))
t[:] = np.interp(h_rad, z_snd - zs_snd, t_snd)
qv = rad_group.createVariable("vapor_mixing_ratio", "f8", ("layers"))
qv[:] = np.interp(h_rad, z_snd - zs_snd, rv_snd.magnitude)
ql = rad_group.createVariable("liquid_mixing_ratio", "f8", ("layers"))
ql[:] = np.zeros(h_rad.shape)
qi = rad_group.createVariable("ice_mixing_ratio", "f8", ("layers"))
qi[:] = np.zeros(h_rad.shape)

In [ ]:
outroot.close()